<a href="https://colab.research.google.com/github/SdmOrthopedic/project-mern/blob/main/autoquoteresponse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import smtplib
import imaplib
import email
from email.mime.text import MIMEText
import logging
import re

# Set up logging
logging.basicConfig(level=logging.DEBUG)

# Create a dictionary with sample product data
data = {
    'product_id': [101, 102, 103],
    'product_name': ['Widget A', 'Widget B', 'Widget C'],
    'price': [19.99, 29.99, 39.99]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
df.to_excel('products.xlsx', index=False)

# Read Excel file
def read_excel(file_path):
    df = pd.read_excel(file_path)
    return df

# Fetch product details from DataFrame
def get_product_details(df, product_id):
    product = df[df['product_id'] == int(product_id)]
    if not product.empty:
        return product.iloc[0]['product_name'], product.iloc[0]['price']
    return None, None

# Connect to the email server and fetch unread emails
def fetch_unread_emails(username, password, imap_server):
    mail = imaplib.IMAP4_SSL(imap_server)
    mail.login(username, password)
    mail.select('inbox')

    status, messages = mail.search(None, '(UNSEEN)')
    email_ids = messages[0].split()

    logging.debug(f"Fetched {len(email_ids)} unread emails")

    emails = []
    for e_id in email_ids:
        status, msg_data = mail.fetch(e_id, '(RFC822)')
        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)
        emails.append(msg)

    return emails

# Extract product ID from email
def extract_product_id(email_msg):
    payload = email_msg.get_payload()
    if email_msg.is_multipart():
        for part in email_msg.walk():
            if part.get_content_type() == 'text/plain':
                payload = part.get_payload(decode=True)
                break
    else:
        payload = email_msg.get_payload(decode=True)

    if payload:
        try:
            payload_str = payload.decode('utf-8').strip()
        except UnicodeDecodeError:
            payload_str = payload.decode('latin1').strip()

        # Extract the product ID using a regex
        match = re.search(r'\b\d+\b', payload_str)
        if match:
            product_id = match.group()
            logging.debug(f"Extracted product ID: {product_id}")
            return product_id
        else:
            logging.error("No valid product ID found in the email body")
            return None
    else:
        logging.error("Payload is None, cannot extract product ID")
        return None

# Send email with product details
def send_email(smtp_server, smtp_port, username, password, to_email, subject, body):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = username
    msg['To'] = to_email

    logging.debug(f"Sending email to {to_email} with subject '{subject}'")

    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(username, password)
        server.sendmail(username, to_email, msg.as_string())

# Main function
def main():
    excel_file_path = 'products.xlsx'
    df = read_excel(excel_file_path)

    email_username = '@gmail.com'
    email_password = ''
    imap_server = 'imap.gmail.com'
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587

    emails = fetch_unread_emails(email_username, email_password, imap_server)

    for email_msg in emails:
        try:
            print("Received Email:")
            print("From:", email_msg['From'])
            print("Subject:", email_msg['Subject'])
            print("Body:", email_msg.get_payload(decode=True).decode('utf-8') if email_msg.get_payload(decode=True) else 'No content')

            product_id = extract_product_id(email_msg)
            if product_id:
                product_name, price = get_product_details(df, product_id)

                if product_name:
                    response_subject = f'Product Details for ID: {product_id}'
                    response_body = f'Product Name: {product_name}\nPrice: {price}'
                else:
                    response_subject = f'Product ID: {product_id} Not Found'
                    response_body = f'No details found for Product ID: {product_id}'

                to_email = email_msg['From']

                print("Sending Email:")
                print("To:", to_email)
                print("Subject:", response_subject)
                print("Body:", response_body)

                send_email(smtp_server, smtp_port, email_username, email_password, to_email, response_subject, response_body)
        except Exception as e:
            logging.error(f"Error processing email from {email_msg['From']}: {e}")

if __name__ == '__main__':
    main()


Received Email:
From: Arshdeep Singh <arshdeepsingh9560@gmail.com>
Subject: andbd
Body: No content
Sending Email:
To: Arshdeep Singh <arshdeepsingh9560@gmail.com>
Subject: Product Details for ID: 103
Body: Product Name: Widget C
Price: 39.99
